# User routes on the site
## Description
**Clickstream** is a sequence of user actions on a website. It allows you to understand how users interact with the site. In this task, you need to find the most frequent custom routes.

## Input data
Input data is а table with clickstream data in file `hdfs:/data/clickstream.csv`.

### Table structure
* `user_id (int)` - Unique user identifier.
* `session_id (int)` - Unique identifier for the user session. The user's session lasts until the identifier changes.
* `event_type (string)` - Event type from the list:
    * **page** - visit to the page
    * **event** - any action on the page
    * <b>&lt;custom&gt;</b> - string with any other type
* `event_type (string)` - Page on the site.
* `timestamp (int)` - Unix-timestamp of action.

### Browser errors
Errors can sometimes occur in the user's browser - after such an error appears, we can no longer trust the data of this session and all the following lines after the error or at the same time with it are considered corrupted and **should not be counted** in statistics.

When an error occurs on the page, a random string containing the word **error** will be written to the `event_type` field.

### Sample of user session
<pre>
+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1620494781|
|    562|       507|       event|      main|1620494788|
|    562|       507|       event|      main|1620494798|
|    562|       507|        page|    family|1620494820|
|    562|       507|       event|    family|1620494828|
|    562|       507|        page|      main|1620494848|
|    562|       507|wNaxLlerrorU|      main|1620494865|
|    562|       507|       event|      main|1620494873|
|    562|       507|        page|      news|1620494875|
|    562|       507|        page|   tariffs|1620494876|
|    562|       507|       event|   tariffs|1620494884|
|    562|       514|        page|      main|1620728918|
|    562|       514|       event|      main|1620729174|
|    562|       514|        page|   archive|1620729674|
|    562|       514|        page|     bonus|1620729797|
|    562|       514|        page|   tariffs|1620731090|
|    562|       514|       event|   tariffs|1620731187|
+-------+----------+------------+----------+----------+
</pre>

#### Correct user routes for a given user:
* **Session 507**: main-family-main
* **Session 514**: main-archive-bonus-tariffs

Route elements are ordered by the time they appear in the clickstream, from earliest to latest.

The route must be accounted for completely before the end of the session or an error in the session.

## Task
You need to use the Spark SQL, Spark RDD and Spark DF interfaces to create a solution file, the lines of which contain **the 30 most frequent user routes** on the site.

Each line of the file should contain the `route` and `count` values **separated by tabs**, where:
* `route` - route on the site, consisting of pages separated by "-".
* `count` - the number of user sessions in which this route was.

The lines must be **ordered in descending order** of the `count` field.

## Criteria
You can get maximum of 3.5 points (final grade) for this assignment, depedning on the number of interface you manage to leverage. The criteria are as follows:

* 0.5 points – Spark SQL solution with 1 query
* 0.5 points – Spark SQL solution with <=2 queries
* 0.5 points – Spark RDD solution
* 0.5 points – Spark DF solution
* 0.5 points – your solution algorithm is relatively optimized, i.e.: no O^2 or O^3 complexities; appropriate object usage; no data leaks etc. This is evaluated by staff.
* 1 point – 1 on 1 screening session. During this session staff member can ask you questions regarding your solution logic, framework usage, questionable parts of your code etc. If your code is clean enough, the staff member can just ask you to solve a theoretical problem connected to Spark.


In [3]:
! hadoop fs -copyFromLocal clickstream.csv /

In [7]:
# Spark SQL 

In [2]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
2023-10-01 14:42:02,710 WARN yarn.Client: Neither spark.yarn.jars nor spark.yarn.archive is set, falling back to uploading libraries under SPARK_HOME.


In [3]:
data = se.read.csv("hdfs:///clickstream.csv", header=True, inferSchema=True, sep = "\t")

In [8]:
data.registerTempTable("df")
data.limit(5).toPandas()

,user_id,session_id,event_type,event_page,timestamp
0,562,507,page,main,1695584127
1,562,507,event,main,1695584134
2,562,507,event,main,1695584144
3,562,507,event,main,1695584147
4,562,507,wNaxLlerrorU,main,1695584154


In [9]:
data.show(10)
print("Data Types:", data.dtypes)

+-------+----------+------------+----------+----------+
|user_id|session_id|  event_type|event_page| timestamp|
+-------+----------+------------+----------+----------+
|    562|       507|        page|      main|1695584127|
|    562|       507|       event|      main|1695584134|
|    562|       507|       event|      main|1695584144|
|    562|       507|       event|      main|1695584147|
|    562|       507|wNaxLlerrorU|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584154|
|    562|       507|       event|      main|1695584160|
|    562|       507|        page|    rabota|1695584166|
|    562|       507|       event|    rabota|1695584174|
+-------+----------+------------+----------+----------+
only showing top 10 rows

Data Types: [('user_id', 'int'), ('session_id', 'int'), ('event_type', 'string'), ('event_page', 'string'), ('timestamp', 'int')]


In [11]:
data.count()

1000000

# SQL

In [6]:
data.createOrReplaceTempView("events")

In [7]:
result_df = se.sql("""
SELECT route, count(route) as cnt FROM (
    SELECT user_id, session_id, CONCAT_WS("-", COLLECT_LIST(event_page)) AS route
    FROM
            (SELECT 
                user_id, 
                session_id, 
                event_type, 
                event_page, 
                timestamp, 
                MAX(CASE WHEN event_type like '%error%' THEN 1 ELSE 0 END) 
                    OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as error_flag
            FROM events)
    WHERE error_flag = 0
    GROUP BY user_id, session_id
)
GROUP BY route
ORDER BY cnt desc
LIMIT 30
""")


In [8]:
result_df.show()

+--------------------+----+
|               route| cnt|
+--------------------+----+
|                main|3434|
|           main-main|1979|
|      main-main-main|1128|
| main-main-main-main| 696|
|main-main-main-ma...| 406|
|main-main-main-ma...| 220|
|        main-archive| 201|
|         main-rabota| 177|
|       main-internet| 158|
|          main-bonus| 153|
|           main-news| 147|
|main-main-main-ma...| 145|
|  main-rabota-rabota| 127|
|        main-tariffs| 117|
|main-archive-archive| 116|
|    main-main-rabota| 111|
|main-internet-int...| 110|
|   main-main-archive| 109|
|         main-online| 106|
|     main-main-bonus|  98|
+--------------------+----+
only showing top 20 rows



## In task we don't have direct rules for different events like main-main, so, the other possible solutuion

In [11]:
result_df = se.sql("""
SELECT route, count(route) as cnt FROM (
    SELECT user_id, session_id, CONCAT_WS("-", COLLECT_LIST(event_page)) AS route
    FROM
            (SELECT distinct
                user_id, 
                session_id,
                event_page, 
                MAX(CASE WHEN event_type like '%error%' THEN 1 ELSE 0 END) 
                    OVER (PARTITION BY user_id, session_id ORDER BY timestamp) as error_flag
            FROM events)
    WHERE error_flag = 0
    GROUP BY user_id, session_id
)
GROUP BY route
ORDER BY cnt desc
LIMIT 30
""")


In [12]:
result_df.show()

+--------------------+----+
|               route| cnt|
+--------------------+----+
|                main|8184|
|        main-archive|1212|
|         main-rabota|1155|
|       main-internet| 980|
|          main-bonus| 941|
|           main-news| 834|
|        main-tariffs| 732|
|         main-online| 635|
|          main-vklad| 549|
| main-archive-rabota| 232|
| main-rabota-archive| 224|
|main-internet-rabota| 189|
|  main-bonus-archive| 186|
|main-archive-inte...| 185|
|   main-bonus-rabota| 185|
|    main-news-rabota| 182|
|   main-rabota-bonus| 179|
|main-rabota-internet| 177|
|   main-archive-news| 173|
|main-internet-arc...| 173|
+--------------------+----+
only showing top 20 rows



I think both cases looks correct in terms of task, will choose the 1st one.
The only difference is **distinct** or not

# DF

In [14]:
from pyspark.sql.window import Window
from pyspark.sql.functions import col, concat_ws, collect_list, max

In [15]:
type(data)

pyspark.sql.dataframe.DataFrame

In [16]:
df = data

In [23]:
w = Window.partitionBy('user_id', 'session_id').orderBy('timestamp')
df = df.withColumn('error_flag', max((col('event_type').like('%error%')).cast('int')).over(w))

In [33]:
df_filtered2 = df.filter(col('error_flag') == 0)
route_df2 = df_filtered2.groupBy('user_id', 'session_id') \
                     .agg(concat_ws('-', collect_list('event_page')).alias('route'))
result_df = route_df2.groupBy('route').count() \
                    .orderBy('count', ascending=False) \
                    .limit(30)
result_df.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 3434|
|           main-main| 1979|
|      main-main-main| 1128|
| main-main-main-main|  696|
|main-main-main-ma...|  406|
|main-main-main-ma...|  220|
|        main-archive|  201|
|         main-rabota|  177|
|       main-internet|  158|
|          main-bonus|  153|
|           main-news|  147|
|main-main-main-ma...|  145|
|  main-rabota-rabota|  127|
|        main-tariffs|  117|
|main-archive-archive|  116|
|    main-main-rabota|  111|
|main-internet-int...|  110|
|   main-main-archive|  109|
|         main-online|  106|
|     main-main-bonus|   98|
+--------------------+-----+
only showing top 20 rows



Just for demonstration

In [29]:
df_filtered = df.select("user_id", "session_id", "event_page").distinct().filter(col('error_flag') == 0)
route_df = df_filtered.groupBy('user_id', 'session_id') \
                     .agg(concat_ws('-', collect_list('event_page')).alias('route'))

In [30]:
result_df = route_df.groupBy('route').count() \
                    .orderBy('count', ascending=False) \
                    .limit(30)

In [31]:
result_df.show()

+--------------------+-----+
|               route|count|
+--------------------+-----+
|                main| 8184|
|        main-archive| 1189|
|         main-rabota| 1143|
|       main-internet|  968|
|          main-bonus|  924|
|           main-news|  817|
|        main-tariffs|  722|
|         main-online|  626|
|          main-vklad|  543|
| main-archive-rabota|  233|
| main-rabota-archive|  220|
|main-internet-rabota|  187|
|main-archive-inte...|  186|
|  main-bonus-archive|  181|
|   main-bonus-rabota|  180|
|   main-rabota-bonus|  178|
|main-rabota-internet|  174|
|   main-archive-news|  171|
|    main-rabota-news|  171|
|    main-news-rabota|  171|
+--------------------+-----+
only showing top 20 rows



# RDD

In [99]:
rdd = sc.textFile("hdfs:///clickstream.csv")

In [100]:
header = rdd.first()
rdd = rdd.filter(lambda line: line != header)

In [101]:
def parse_csv_line(line):
    return line.split('\t')

In [102]:
parsed_rdd = rdd.map(parse_csv_line)

In [103]:
parsed_rdd.take(5)

[['562', '507', 'page', 'main', '1695584127'],
 ['562', '507', 'event', 'main', '1695584134'],
 ['562', '507', 'event', 'main', '1695584144'],
 ['562', '507', 'event', 'main', '1695584147'],
 ['562', '507', 'wNaxLlerrorU', 'main', '1695584154']]

In [116]:
def add_error_flag(row):
    user_id, session_id, event_type, event_page, timestamp = row
    return (user_id, session_id, event_type, event_page, timestamp, 1 if 'error' in event_type else 0)

In [117]:
events_with_error_flag = parsed_rdd.map(add_error_flag)

In [120]:
events_filtered = events_with_error_flag.filter(lambda x: x[5] == 0)

In [121]:
events_filtered.take(5)

[('562', '507', 'page', 'main', '1695584127', 0),
 ('562', '507', 'event', 'main', '1695584134', 0),
 ('562', '507', 'event', 'main', '1695584144', 0),
 ('562', '507', 'event', 'main', '1695584147', 0),
 ('562', '507', 'event', 'main', '1695584154', 0)]

In [128]:
grouped_events = events_filtered.groupBy(lambda x: (x[0], x[1]))

In [129]:
def collect_routes(grouped):
    (user_id, session_id), events = grouped
    pages = [event[3] for event in events]
    route = '-'.join(pages)
    return ((user_id, session_id), route)

In [130]:
routes_rdd = grouped_events.map(collect_routes)

In [131]:
counted_routes = routes_rdd.map(lambda x: (x[1], 1)).reduceByKey(lambda x, y: x + y)

In [132]:
top_30_routes = counted_routes.sortBy(lambda x: x[1], ascending=False).take(30)

In [133]:
for route, count in top_30_routes:
    print(f"Route: {route}, Count: {count}")

Route: main, Count: 2435
Route: main-main, Count: 1459
Route: main-main-main, Count: 850
Route: main-main-main-main, Count: 549
Route: main-main-main-main-main, Count: 321
Route: main-main-main-main-main-main, Count: 168
Route: main-archive, Count: 153
Route: main-rabota, Count: 145
Route: main-internet, Count: 122
Route: main-main-main-main-main-main-main, Count: 118
Route: main-news, Count: 102
Route: main-bonus, Count: 99
Route: main-rabota-rabota, Count: 92
Route: main-main-rabota, Count: 86
Route: main-main-archive, Count: 84
Route: main-archive-archive, Count: 84
Route: main-tariffs, Count: 80
Route: main-online, Count: 77
Route: main-internet-internet, Count: 73
Route: main-main-bonus, Count: 73
Route: main-vklad, Count: 72
Route: main-main-news, Count: 68
Route: main-main-main-archive, Count: 65
Route: main-bonus-bonus, Count: 62
Route: main-main-main-main-main-main-main-main, Count: 58
Route: main-main-online, Count: 55
Route: main-news-news, Count: 53
Route: main-bonus-bonus-

In [134]:
se.stop()